# Get Change in Estimate for Incremented Numerosity for Classifier Networks

Select the model, and make sure that the batch_size in model_settings.py is 1000 (the number of sets). Make sure analysis_estimation_nds.py imports FF_estimation.py.

In [1]:
import numpy as np
import plotly.graph_objs as go
import importlib
import analysis_estimation_nds
importlib.reload(analysis_estimation_nds)
import create_data_po
importlib.reload(create_data_po)
from jupyterthemes import jtplot
jtplot.style()
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

['/home/sychen23/.local/lib/python3.5/site-packages/ipykernel_launcher.py', '-f', 'true', 'true', 'true', 'true', 'true', 'model_runs/-f/classify_log.csv', 'model_runs/-f/classifymodel_0.ckpt', 'model_runs/-f/classifymodel_', 'model_runs/-f/zzzdraw_data_5000.npy', 'false', 'true', 'false', 'false', 'true']
Instructions for updating:
keep_dims is deprecated, use keepdims instead
analysis_estimation_nds.py
analysis_estimation_nds.py


In [2]:
models = [("New_DAA_decrs_fN", "$f(n) \propto 1/n^2$, Decr Avg Area"),
          ("New_DAA_const_fN", "$f(n) \propto 1/n^2$, Const Avg Area"),
          ("New_CAA_decrs_fN", "$f(n)$ Constant, Decr Avg Area"),
          ("New_CAA_const_fN", "$f(n)$ Constant, Const Avg Area")]
model = models[3]
model_name, model_figure_title = model
num_runs = 10
iter_list = [0, 200, 400, 1600, 6400, 25600, 102400, 204800, 409600, 819200, 1000000, 1228800, 1638400, 2000000, 3000000]#, 4000000, 5000000, 6000000]
train_max_blobs = 15
test_max_blobs = 9
train_min_blobs = test_min_blobs = 1

num_iters = len(iter_list)
num_sets = 1000
train_output_size = train_max_blobs - train_min_blobs + 1
test_output_size = test_max_blobs - test_min_blobs + 1
test_condition = "po_inc"
data_directory = "data/" + model_name + "/" + test_condition + "/"

import os
if not os.path.exists(data_directory):
    os.makedirs(data_directory)
    
scalar = False

In [3]:
def get_classifications_one_run(path, iteration):
    """Get classifications for one run at one iteration for all 1000 sets."""
    imgs_data = analysis_estimation_nds.classify_imgs_po_sets(iteration, True, path=path, scalar=scalar)
    confidence_one_run = np.zeros([num_sets, train_output_size, train_output_size])
    choice_one_run = np.zeros([num_sets, train_output_size, train_output_size])

    for i in range(num_sets):
        for nb in range(test_output_size): 
            num_blobs = nb + test_min_blobs
            confidence_hist = np.zeros(train_output_size)
            choice_hist = np.zeros(train_output_size)
            num_imgs_with_num_blobs = 0.00001

            for idx, data in enumerate(imgs_data[i*test_output_size:i*test_output_size+test_output_size]):
                if data["label"][nb] == 1: # data is for an image with num_blobs blobs
                    num_imgs_with_num_blobs += 1

                    # Histogram of softmaxes
                    confidence_hist += np.array(data["classifications"][0])

                    # Histogram of choices
                    choice = np.argmax(data["classifications"][0])
                    choice_list = [0] * train_output_size
                    choice_list[choice] = 1
                    choice_hist += choice_list

            confidence_hist = confidence_hist / num_imgs_with_num_blobs
            confidence_one_run[i, nb] = confidence_hist.tolist()
            choice_hist = choice_hist / num_imgs_with_num_blobs
            choice_one_run[i, nb] = choice_hist.tolist()
    return confidence_one_run, choice_one_run

In [4]:
# Test out one iteration
path = 'model_runs/' + model_name + '_run' + str(1)
imgs_data = analysis_estimation_nds.classify_imgs_po_sets(0, True, path=path, scalar=scalar)
print(np.shape(imgs_data[0]['label']))
print([imgs_data[i]['label'] for i in range(0,9)]) # 0 to 8 make up first set, numerosities from 9 to 1.
print(imgs_data[8]['classifications'][0]) # softmax output

INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run1/classifymodel_0.ckpt
(9,)
[array([0., 0., 0., 0., 0., 0., 0., 0., 1.]), array([0., 0., 0., 0., 0., 0., 0., 1., 0.]), array([0., 0., 0., 0., 0., 0., 1., 0., 0.]), array([0., 0., 0., 0., 0., 1., 0., 0., 0.]), array([0., 0., 0., 0., 1., 0., 0., 0., 0.]), array([0., 0., 0., 1., 0., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0.])]
[0.0654109  0.13816643 0.06365746 0.02140633 0.03076288 0.01454673
 0.0360908  0.11460319 0.01088589 0.01558783 0.03701327 0.05444406
 0.28588018 0.05642338 0.05512072]


In [5]:
i = 2
[x['label'] for x in imgs_data[i*test_output_size:i*test_output_size+test_output_size]]

[array([0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0.]),
 array([0., 0., 0., 0., 0., 0., 1., 0., 0.]),
 array([0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0.])]

In [6]:
confidence_all_runs = np.zeros([num_runs, num_iters, num_sets, train_output_size, train_output_size]) 
choice_all_runs = np.zeros([num_runs, num_iters, num_sets, train_output_size, train_output_size])
for run in range(num_runs):
    path = 'model_runs/' + model_name + '_run' + str(run + 1) # '/run_' if all the runs are stored in one folder
    for i, iteration in enumerate(iter_list):
        confidence_all_runs[run, i], choice_all_runs[run, i] = get_classifications_one_run(path, iteration)

np.save(data_directory + "confidence_hist_set", confidence_all_runs)
np.save(data_directory + "choice_hist_set", choice_all_runs)

INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run1/classifymodel_0.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run1/classifymodel_200.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run1/classifymodel_400.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run1/classifymodel_1600.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run1/classifymodel_6400.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run1/classifymodel_25600.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run1/classifymodel_102400.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run1/classifymodel_204800.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run1/classifymodel_409600.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run1/classifymodel_819200.ckpt
INFO:t

INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run6/classifymodel_204800.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run6/classifymodel_409600.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run6/classifymodel_819200.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run6/classifymodel_1000000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run6/classifymodel_1228800.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run6/classifymodel_1638400.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run6/classifymodel_2000000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run6/classifymodel_3000000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run7/classifymodel_0.ckpt
INFO:tensorflow:Restoring parameters from model_runs/New_CAA_const_fN_run7/classifymodel_20

In [7]:
def get_msdcv(hist_matrix):
    """Get the mean, standard deviation, and coefficient of variation matrices from histogram matrix."""
    
    msdcv = np.zeros([num_runs, num_iters, num_sets, train_output_size, 3])
    
    for run in range(num_runs):
        for i, it in enumerate(iter_list):
            for s in range(num_sets):
                for t in range(train_output_size):
                    values_sum = 0
                    sqr_sum = 0

                    # Find the mean
                    for p in range(train_output_size):
                        values_sum += (p + 1) * hist_matrix[run, i, s, t, p]
                    msdcv[run, i, s, t, 0] = mu = values_sum

                    # Find the standard deviation
                    for p in range(train_output_size):
                        sqr_sum += (p + 1 - mu)**2 * hist_matrix[run, i, s, t, p]
                    msdcv[run, i, s, t, 1] = sigma = np.sqrt(sqr_sum)

                    # Find the coefficient of variation
                    msdcv[run, i, s, t, 2] = cv = sigma / mu
    return np.mean(msdcv, axis=0)

In [8]:
print("Confidence")
confidence_all_runs = np.load(data_directory + 'confidence_hist_set.npy') # runs, iters, sets, inputs, outputs
confidence_msdcv = get_msdcv(confidence_all_runs)
d_confidence = np.diff(np.mean(confidence_msdcv, axis=1)[-1,:9,0])
print(d_confidence)
print('mean of d_confidence: %f' % np.mean(d_confidence))
print('sd of d_confidence: %f' % np.std(d_confidence))

Confidence
[1.03043266 1.07736697 1.15685388 1.12511279 1.18483044 1.17006965
 1.17966557 1.19051028]
mean of d_confidence: 1.139355
sd of d_confidence: 0.054163


In [9]:
print("Choice")
choice_all_runs = np.load(data_directory + 'choice_hist_set.npy') # runs, iters, sets, inputs, outputs
choice_msdcv = get_msdcv(choice_all_runs)
d_choice = np.diff(np.mean(choice_msdcv, axis=1)[-1,:9,0])
print(d_choice)
print('mean of d_choice: %f' % np.mean(d_choice))
print('sd of d_choice: %f' % np.std(d_choice))

Choice
[1.00878991 1.05428946 1.15848842 1.11948881 1.19158808 1.14388856
 1.16368836 1.1901881 ]
mean of d_choice: 1.128801
sd of d_choice: 0.061317
